In [1]:
import atoti as tt
from pathlib import Path

In [2]:
session = tt.Session.start()

In [3]:
data_source = "/home/olivier/projects/activeviam/gits/git-stats/target/files"

In [4]:
commit_table = session.read_parquet(
    Path(data_source) / "commits-*.parquet",
    table_name="commits")

In [5]:
branch_table = session.read_parquet(
    Path(data_source) / "branch-*.parquet",
    table_name="branch",
    keys={"commit", "branch"})

In [6]:
change_table = session.read_parquet(
    Path(data_source) / "changes-*.parquet",
    table_name="changes",
    keys={"commit", "path"})

In [7]:
renaming_table = session.read_parquet(
    Path(data_source) / "renaming-*.parquet",
    table_name="renamings",
    keys={"commit", "timestamp", "before", "after"}
)

In [14]:
session.tables.schema

```mermaid
erDiagram
  "commits" {
    non-null String "commit"
    nullable long "timestamp"
  }
  "branch" {
    non-null String PK "branch"
    non-null String PK "commit"
  }
  "changes" {
    non-null String PK "commit"
    non-null String "module"
    non-null String "filename"
    non-null String PK "path"
    nullable long "additions"
    nullable long "deletions"
  }
  "renamings" {
    non-null String PK "commit"
    non-null long PK "timestamp"
    non-null String PK "before"
    non-null String PK "after"
  }
  "commits" }o..o{ "branch" : "commit == commit"
  "changes" }o--o| "commits" : "commit == commit"
  "changes" }o..o{ "renamings" : "(commit == commit) & (path == after)"
```


In [9]:
change_table.head()

,,module,filename,additions,deletions
commit,path,,,,
65bef2269cf7e526f9bcde4fbebe0c64c36964c7,database/clickhouse/src/main/java/com/activeviam/database/clickhouse/private_/query/ClickhouseExecutableQueryBuilder.java,database/clickhouse,ClickhouseExecutableQueryBuilder.java,32,11
ea34b34fd65f336be93cb9a26099cc575c1a5d79,pivot-server/sql-bridge/src/test/java/com/activeviam/activepivot/server/sql/internal/TestPowerBiDiscovery.java,pivot-server/sql-bridge,TestPowerBiDiscovery.java,77,0
65bef2269cf7e526f9bcde4fbebe0c64c36964c7,database/clickhouse/src/test/java/com/activeviam/database/clickhouse/private_/query/TestClickhouseV21SqlJoinCompiler.java,database/clickhouse,TestClickhouseV21SqlJoinCompiler.java,3,1
215f62650e7449ac3ce30d490354ced6d7e96a5d,pivot-server/spring/src/test/java/com/quartetfs/biz/jettyserver/DatastoreRestServicesConfigWithAuthForTest.java,pivot-server/spring,DatastoreRestServicesConfigWithAuthForTest.java,35,0
fe870a2b49f64b96f12fa7f107a3354f872fe38d,datastore/core/src/main/java/com/qfs/store/query/impl/CompiledStoreSearch.java,datastore/core,CompiledStoreSearch.java,1,1


In [10]:
commit_table.row_count

9985

In [11]:
commit_table.join(branch_table, commit_table["commit"] == branch_table["commit"])

In [12]:
change_table.join(commit_table, change_table["commit"] == commit_table["commit"])

In [13]:
change_table.join(renaming_table, (change_table["commit"] == renaming_table["commit"]) & (change_table["path"] == renaming_table["after"]))

In [15]:
cube = session.create_cube(name="git", fact_table=change_table)

In [16]:
session.url


'http://localhost:33501'